In [ ]:
# https://github.com/qubvel/segmentation_models
!pip install segmentation-models

In [ ]:
"""replace this with segmentation_models.models.unet.py and restart runtime"""
# from keras_applications import get_submodules_from_kwargs

# from ._common_blocks import Conv2dBn
# from ._utils import freeze_model
# from ..backbones.backbones_factory import Backbones

# backend = None
# layers = None
# models = None
# keras_utils = None


# # ---------------------------------------------------------------------
# #  Utility functions
# # ---------------------------------------------------------------------

# def get_submodules():
#     return {
#         'backend': backend,
#         'models': models,
#         'layers': layers,
#         'utils': keras_utils,
#     }


# # ---------------------------------------------------------------------
# #  Blocks
# # ---------------------------------------------------------------------

# def Conv3x3BnReLU(filters, use_batchnorm, name=None):
#     kwargs = get_submodules()

#     def wrapper(input_tensor):
#         return Conv2dBn(
#             filters,
#             kernel_size=3,
#             activation='relu',
#             kernel_initializer='he_uniform',
#             padding='same',
#             use_batchnorm=use_batchnorm,
#             name=name,
#             **kwargs
#         )(input_tensor)

#     return wrapper


# def DecoderUpsamplingX2Block(filters, decoder_num, stage, use_batchnorm=False):
#     up_name = 'decoder{}_stage{}_upsampling'.format(decoder_num, stage)
#     conv1_name = 'decoder{}_stage{}a'.format(decoder_num, stage)
#     conv2_name = 'decoder{}_stage{}b'.format(decoder_num, stage)
#     concat_name = 'decoder{}_stage{}_concat'.format(decoder_num, stage)

#     concat_axis = 3 if backend.image_data_format() == 'channels_last' else 1

#     def wrapper(input_tensor, skip=None):
#         x = layers.UpSampling2D(size=2, name=up_name)(input_tensor)

#         if skip is not None:
#             x = layers.Concatenate(axis=concat_axis, name=concat_name)([x, skip])

#         x = Conv3x3BnReLU(filters, use_batchnorm, name=conv1_name)(x)
#         x = Conv3x3BnReLU(filters, use_batchnorm, name=conv2_name)(x)

#         return x

#     return wrapper


# def DecoderTransposeX2Block(filters, decoder_num, stage, use_batchnorm=False):
#     transp_name = 'decoder{}_stage{}a_transpose'.format(decoder_num, stage)
#     bn_name = 'decoder{}_stage{}a_bn'.format(decoder_num, stage)
#     relu_name = 'decoder{}_stage{}a_relu'.format(decoder_num, stage)
#     conv_block_name = 'decoder{}_stage{}b'.format(decoder_num, stage)
#     concat_name = 'decoder{}_stage{}_concat'.format(decoder_num, stage)

#     concat_axis = bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

#     def layer(input_tensor, skip=None):

#         x = layers.Conv2DTranspose(
#             filters,
#             kernel_size=(4, 4),
#             strides=(2, 2),
#             padding='same',
#             name=transp_name,
#             use_bias=not use_batchnorm,
#         )(input_tensor)

#         if use_batchnorm:
#             x = layers.BatchNormalization(axis=bn_axis, name=bn_name)(x)

#         x = layers.Activation('relu', name=relu_name)(x)

#         if skip is not None:
#             x = layers.Concatenate(axis=concat_axis, name=concat_name)([x, skip])

#         x = Conv3x3BnReLU(filters, use_batchnorm, name=conv_block_name)(x)

#         return x

#     return layer


# # ---------------------------------------------------------------------
# #  Unet Decoder
# # ---------------------------------------------------------------------

# def build_unet(
#         backbone,
#         decoder_block,
#         skip_connection_layers,
#         decoder_filters=(256, 128, 64, 32, 16),
#         n_upsample_blocks=5,
#         classes=1,
#         activation='sigmoid',
#         use_batchnorm=True,
# ):
#     input_ = backbone.input
#     x1 = backbone.output
#     x2 = backbone.output

#     # extract skip connections
#     skips = ([backbone.get_layer(name=i).output if isinstance(i, str)
#               else backbone.get_layer(index=i).output for i in skip_connection_layers])

#     # add center block if previous operation was maxpooling (for vgg models)
#     if isinstance(backbone.layers[-1], layers.MaxPooling2D):
#         x1 = Conv3x3BnReLU(512, use_batchnorm, name='center_block1_1')(x1)
#         x1 = Conv3x3BnReLU(512, use_batchnorm, name='center_block2_1')(x1)

#     # add center block if previous operation was maxpooling (for vgg models)
#     if isinstance(backbone.layers[-1], layers.MaxPooling2D):
#         x2 = Conv3x3BnReLU(512, use_batchnorm, name='center_block1_2')(x2)
#         x2 = Conv3x3BnReLU(512, use_batchnorm, name='center_block2_2')(x2)

#     # building decoder blocks
#     for i in range(n_upsample_blocks):

#         if i < len(skips):
#             skip = skips[i]
#         else:
#             skip = None

#         x1 = decoder_block(decoder_filters[i], decoder_num=1, stage=i, use_batchnorm=use_batchnorm)(x1, skip)

#     # building decoder blocks
#     for i in range(n_upsample_blocks):

#         if i < len(skips):
#             skip = skips[i]
#         else:
#             skip = None

#         x2 = decoder_block(decoder_filters[i], decoder_num=2, stage=i, use_batchnorm=use_batchnorm)(x2, skip)


#     # model head (define number of output classes)
#     x1 = layers.Conv2D(
#         filters=1,
#         kernel_size=(3, 3),
#         padding='same',
#         use_bias=True,
#         kernel_initializer='glorot_uniform',
#         name='final_conv_1',
#     )(x1)
#     x1 = layers.Activation("sigmoid", name="Binary_segmentation")(x1)

#     x2 = layers.Conv2D(
#         filters=3,
#         kernel_size=(3, 3),
#         padding='same',
#         use_bias=True,
#         kernel_initializer='glorot_uniform',
#         name='final_conv_2',
#     )(x2)
#     x2 = layers.Activation("softmax", name="MultiClass_segmentation")(x2)

#     # create keras model instance
#     model = models.Model(input_, [x1, x2])

#     return model


# # ---------------------------------------------------------------------
# #  Unet Model
# # ---------------------------------------------------------------------

# def Unet(
#         backbone_name='vgg16',
#         input_shape=(None, None, 3),
#         classes=1,
#         activation='sigmoid',
#         weights=None,
#         encoder_weights='imagenet',
#         encoder_freeze=False,
#         encoder_features='default',
#         decoder_block_type='upsampling',
#         decoder_filters=(256, 128, 64, 32, 16),
#         decoder_use_batchnorm=True,
#         **kwargs
# ):
#     """ Unet_ is a fully convolution neural network for image semantic segmentation

#     Args:
#         backbone_name: name of classification model (without last dense layers) used as feature
#             extractor to build segmentation model.
#         input_shape: shape of input data/image ``(H, W, C)``, in general
#             case you do not need to set ``H`` and ``W`` shapes, just pass ``(None, None, C)`` to make your model be
#             able to process images af any size, but ``H`` and ``W`` of input images should be divisible by factor ``32``.
#         classes: a number of classes for output (output shape - ``(h, w, classes)``).
#         activation: name of one of ``keras.activations`` for last model layer
#             (e.g. ``sigmoid``, ``softmax``, ``linear``).
#         weights: optional, path to model weights.
#         encoder_weights: one of ``None`` (random initialization), ``imagenet`` (pre-training on ImageNet).
#         encoder_freeze: if ``True`` set all layers of encoder (backbone model) as non-trainable.
#         encoder_features: a list of layer numbers or names starting from top of the model.
#             Each of these layers will be concatenated with corresponding decoder block. If ``default`` is used
#             layer names are taken from ``DEFAULT_SKIP_CONNECTIONS``.
#         decoder_block_type: one of blocks with following layers structure:

#             - `upsampling`:  ``UpSampling2D`` -> ``Conv2D`` -> ``Conv2D``
#             - `transpose`:   ``Transpose2D`` -> ``Conv2D``

#         decoder_filters: list of numbers of ``Conv2D`` layer filters in decoder blocks
#         decoder_use_batchnorm: if ``True``, ``BatchNormalisation`` layer between ``Conv2D`` and ``Activation`` layers
#             is used.

#     Returns:
#         ``keras.models.Model``: **Unet**

#     .. _Unet:
#         https://arxiv.org/pdf/1505.04597

#     """

#     global backend, layers, models, keras_utils
#     backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

#     if decoder_block_type == 'upsampling':
#         decoder_block = DecoderUpsamplingX2Block
#     elif decoder_block_type == 'transpose':
#         decoder_block = DecoderTransposeX2Block
#     else:
#         raise ValueError('Decoder block type should be in ("upsampling", "transpose"). '
#                          'Got: {}'.format(decoder_block_type))

#     backbone = Backbones.get_backbone(
#         backbone_name,
#         input_shape=input_shape,
#         weights=encoder_weights,
#         include_top=False,
#         **kwargs,
#     )

#     if encoder_features == 'default':
#         encoder_features = Backbones.get_feature_layers(backbone_name, n=4)

#     model = build_unet(
#         backbone=backbone,
#         decoder_block=decoder_block,
#         skip_connection_layers=encoder_features,
#         decoder_filters=decoder_filters,
#         classes=classes,
#         activation=activation,
#         n_upsample_blocks=len(decoder_filters),
#         use_batchnorm=decoder_use_batchnorm,
#     )

#     # lock encoder weights for fine-tuning
#     if encoder_freeze:
#         freeze_model(backbone, **kwargs)

#     # loading model weights
#     if weights is not None:
#         model.load_weights(weights)

#     return model

In [ ]:
import segmentation_models as sm
sm.set_framework("tf.keras")
sm.framework()

# Train U-Net with MoNuSeg dataset

In [ ]:
!nvidia-smi

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!python --version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/nuclei_segmentation

In [ ]:
# https://github.com/dovahcrow/patchify.py
!pip install patchify

## Make validation set

In [ ]:
import os
import glob
import random
import shutil
import numpy as np

In [ ]:
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
train_dir = "dataset/monuseg/stain_normalized/train"
val_dir = "dataset/monuseg/stain_normalized/validation"

In [ ]:
def make_validation_set(val_dir, train_dir, val_size=6, seed=42, fold=None):

    create_path(val_dir)
    create_path(os.path.join(val_dir, "tissue_images"))
    create_path(os.path.join(val_dir, "binary_masks"))
    create_path(os.path.join(val_dir, "instance_masks"))
    create_path(os.path.join(val_dir, "modified_masks"))
    
    for j in sorted(glob.glob(os.path.join(val_dir, "tissue_images", "*"))):
        try:
            shutil.move(j, os.path.join(train_dir, "tissue_images"))
            shutil.move(j.replace("tissue_images", "binary_masks").replace("tif", "png"), 
                        os.path.join(train_dir, "binary_masks"))
            shutil.move(j.replace("tissue_images", "instance_masks").replace("tif", "npy"), 
                        os.path.join(train_dir, "instance_masks"))
            shutil.move(j.replace("tissue_images", "modified_masks").replace("tif", "png"), 
                        os.path.join(train_dir, "modified_masks"))
        except:
            continue

    images_lst = sorted(glob.glob(os.path.join(train_dir, "tissue_images", "*")))
    np.random.seed(seed)
    np.random.shuffle(images_lst)
    if fold is None:
        random.seed(seed)
        val_lst = random.sample(images_lst, val_size)
    else:
        val_lst = images_lst[(fold*val_size)-val_size: fold*val_size]
        

    for i in val_lst:
        shutil.move(i, os.path.join(val_dir, "tissue_images"))
        shutil.move(i.replace("tissue_images", "binary_masks").replace("tif", "png"), 
                    os.path.join(val_dir, "binary_masks"))
        shutil.move(i.replace("tissue_images", "instance_masks").replace("tif", "npy"), 
                    os.path.join(val_dir, "instance_masks"))
        shutil.move(i.replace("tissue_images", "modified_masks").replace("tif", "png"), 
                    os.path.join(val_dir, "modified_masks"))
        
    print(f"Validation list: {[os.path.basename(i) for i in val_lst]}")

In [ ]:
make_validation_set(val_dir, train_dir, val_size=6, fold=1)

## read the tissue images & GTs

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify

In [ ]:
print(cv2.__version__)

In [ ]:
train_dir = "dataset/monuseg/stain_normalized/train/tissue_images"
train_maskdir = "dataset/monuseg/stain_normalized/train/binary_masks"
train_mask2dir = "dataset/monuseg/stain_normalized/train/modified_masks"

val_dir   = "dataset/monuseg/stain_normalized/validation/tissue_images"
val_maskdir = "dataset/monuseg/stain_normalized/validation/binary_masks"
val_mask2dir = "dataset/monuseg/stain_normalized/validation/modified_masks"

In [ ]:
W = 1024
H = 1024

patch_size = (256, 256, 3)
all_img_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(train_dir, "*"))), total=len(os.listdir(train_dir))):
    single_img = cv2.imread(x, cv2.IMREAD_COLOR)
    single_img = cv2.cvtColor(single_img, cv2.COLOR_BGR2RGB)
    single_img = cv2.resize(single_img, (W, H), interpolation=cv2.INTER_LINEAR)
    # patchify
    single_img_patches = patchify(single_img, patch_size=patch_size, step=128)
    # squeeze
    single_img_patches = np.squeeze(single_img_patches)

    for i in range(single_img_patches.shape[0]):
        for j in range(single_img_patches.shape[1]):
            all_img_patches.append(single_img_patches[i, j])
    
train_images = np.array(all_img_patches)

In [ ]:
train_images.shape

In [ ]:
all_img_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(val_dir, "*"))), total=len(os.listdir(val_dir))):
    single_img = cv2.imread(x, cv2.IMREAD_COLOR)
    single_img = cv2.cvtColor(single_img, cv2.COLOR_BGR2RGB)
    single_img = cv2.resize(single_img, (W, H), interpolation=cv2.INTER_LINEAR)
    # patchify
    single_img_patches = patchify(single_img, patch_size=patch_size, step=128)
    # squeeze
    single_img_patches = np.squeeze(single_img_patches)

    for i in range(single_img_patches.shape[0]):
        for j in range(single_img_patches.shape[1]):
            all_img_patches.append(single_img_patches[i, j])

val_images = np.array(all_img_patches)

In [ ]:
val_images.shape

In [ ]:
all_mask_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(train_maskdir, "*"))), total=len(os.listdir(train_maskdir))):
    single_mask = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    single_mask = cv2.resize(single_mask, (W, H), interpolation=cv2.INTER_NEAREST)
    # patchify
    single_mask_patches = patchify(single_mask, patch_size=(256, 256), step=128)
    # squeeze
    single_mask_patches = np.squeeze(single_mask_patches)

    for i in range(single_mask_patches.shape[0]):
        for j in range(single_mask_patches.shape[1]):
            all_mask_patches.append(single_mask_patches[i, j])

train_masks = np.array(all_mask_patches)

In [ ]:
train_masks.shape

In [ ]:
all_mask_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(val_maskdir, "*"))), total=len(os.listdir(val_maskdir))):
    single_mask = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    single_mask = cv2.resize(single_mask, (W, H), interpolation=cv2.INTER_NEAREST)
    # patchify
    single_mask_patches = patchify(single_mask, patch_size=(256, 256), step=128)
    # squeeze
    single_mask_patches = np.squeeze(single_mask_patches)

    for i in range(single_mask_patches.shape[0]):
        for j in range(single_mask_patches.shape[1]):
            all_mask_patches.append(single_mask_patches[i, j])

val_masks = np.array(all_mask_patches)

In [ ]:
val_masks.shape

In [ ]:
all_mask_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(train_mask2dir, "*"))), total=len(os.listdir(train_mask2dir))):
    single_mask = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    single_mask = cv2.resize(single_mask, (W, H), interpolation=cv2.INTER_NEAREST)
    # patchify
    single_mask_patches = patchify(single_mask, patch_size=(256, 256), step=128)
    # squeeze
    single_mask_patches = np.squeeze(single_mask_patches)

    for i in range(single_mask_patches.shape[0]):
        for j in range(single_mask_patches.shape[1]):
            # mask_with_boarders = generate_boarder(single_mask_patches[i, j])
            all_mask_patches.append(single_mask_patches[i, j])

train_masks2 = np.array(all_mask_patches)

In [ ]:
train_masks2.shape

In [ ]:
all_mask_patches = []
for x in tqdm(sorted(glob.glob(os.path.join(val_mask2dir, "*"))), total=len(os.listdir(val_mask2dir))):
    single_mask = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    single_mask = cv2.resize(single_mask, (W, H), interpolation=cv2.INTER_NEAREST)
    # patchify
    single_mask_patches = patchify(single_mask, patch_size=(256, 256), step=128)
    # squeeze
    single_mask_patches = np.squeeze(single_mask_patches)

    for i in range(single_mask_patches.shape[0]):
        for j in range(single_mask_patches.shape[1]):
            # mask_with_boarders = generate_boarder(single_mask_patches[i, j])
            all_mask_patches.append(single_mask_patches[i, j])

val_masks2 = np.array(all_mask_patches)

In [ ]:
val_masks2.shape

In [ ]:
# sanity check
rnd = np.random.randint(len(train_images))
# rnd = 222

fig, ax = plt.subplots(1, 3, figsize=(12, 6))
[axi.set_axis_off() for axi in ax.ravel()]

ax[0].imshow(train_images[rnd])
ax[0].set_title("Tissue Image")

ax[1].imshow(train_masks[rnd])
ax[1].set_title("Mask")

# ax[2].imshow(train_images[rnd])
ax[2].imshow(train_masks2[rnd])
ax[2].set_title("Mask2")

plt.tight_layout()
plt.show()

## One-hot encoding the modified masks

In [ ]:
# train masks
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks2.reshape(-1,)
train_masks_reshaped_encoded = label_encoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

n_classes = 3
train_masks_cat = to_categorical(train_masks_encoded_original_shape, num_classes=n_classes)
train_masks_cat.shape

In [ ]:
# val masks
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
n, h, w = val_masks.shape
val_masks_reshaped = val_masks2.reshape(-1,)
val_masks_reshaped_encoded = label_encoder.fit_transform(val_masks_reshaped)
val_masks_encoded_original_shape = val_masks_reshaped_encoded.reshape(n, h, w)

n_classes = 3
val_masks_cat = to_categorical(val_masks_encoded_original_shape, num_classes=n_classes)
val_masks_cat.shape

## Data augmentation using albumentations library

In [ ]:
import albumentations as A
from keras.utils import Sequence

In [ ]:
print(A.__version__)

In [ ]:
import time

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, images, masks, masks_cat, augmentations=None, batch_size=8, img_size=256, n_channels=3, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        
        self.images = images
        self.masks = masks
        self.masks_cat = masks_cat

        self.img_size = img_size
        
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.augment = augmentations
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indices of the batch
        indices = self.indices[index * self.batch_size: min((index + 1) * self.batch_size, len(self.images))]

        # Generate data
        X, y = self.data_generation(indices)
        y1 = y[0]
        y2 = y[1]

        if self.augment is None:
            return X, [np.array(y1), np.array(y2)]
        else:            
            im, mask1, mask2 = [], [], []   
            for x, y1, y2 in zip(X, y1, y2):
                augmented = self.augment(image=x, mask1=y1, mask2=y2)
                im.append(augmented['image'])
                mask1.append(augmented['mask1'])
                mask2.append(augmented['mask2'])

            return np.array(im), [np.array(mask1), np.array(mask2)]

    def on_epoch_end(self):
        'Updates indices after each epoch'
        self.indices = np.arange(len(self.images))
        if self.shuffle == True:
            t = 1000 * time.time() # current time in milliseconds
            np.random.seed(int(t) % 2**32)
            np.random.shuffle(self.indices)

    def data_generation(self, indices):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((len(indices), self.img_size, self.img_size, self.n_channels))
        y1 = np.empty((len(indices), self.img_size, self.img_size, 1))
        y2 = np.empty((len(indices), self.img_size, self.img_size, 3)) # 3 classes (Nuclei, Border, Background)
        # Generate data
        for n, i in enumerate(indices):
            X[n] = self.images[i]
            y1[n] = (self.masks[i]/255.)[..., np.newaxis]
            y2[n] = self.masks_cat[i]

        return np.uint8(X), [np.float32(y1), np.float32(y2)]

In [ ]:
AUGMENTATIONS_TRAIN = A.Compose([
    A.Rotate(limit=360, p=0.5),
    A.OneOf([
        A.HorizontalFlip(),
        A.VerticalFlip(),
        ], p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.RandomGamma(),
        A.GaussNoise()
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        # A.ElasticTransform(alpha=3, sigma=150, alpha_affine=150),
        # A.ElasticTransform(),
        A.Affine(translate_percent=0.2, shear=30, mode=cv2.BORDER_CONSTANT),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.OneOf([
        A.RGBShift(r_shift_limit=40, g_shift_limit=40,  b_shift_limit=40),
        A.ColorJitter(hue=0.1),
        A.Blur(blur_limit=3)
        ], p=0.3),
    A.ToFloat(max_value=255)
], p=1,
additional_targets={'image': 'image', 'mask1': 'mask', 'mask2':'mask'})

AUGMENTATIONS_VAL = A.Compose([
    A.ToFloat(max_value=255)
], p=1,
additional_targets={'image': 'image', 'mask1': 'mask', 'mask2':'mask'})

### Testing data generator

In [ ]:
# Single tissue image with 256*256 tiles (50% overlap between tiles) without augmentation
a = DataGenerator(train_images, train_masks, train_masks_cat, batch_size=49, augmentations=None, shuffle=False)
images, masks = a.__getitem__(0)

max_images = 49
grid_width = 7
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*2, grid_height*2))

for i,(im, mask1, mask2) in enumerate(zip(images, masks[0], masks[1])):
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(im)
    # ax.imshow(mask1.squeeze(), alpha=0.4)
    ax.imshow(mask2, alpha=0.4)
    ax.axis('off')

print(mask1.shape)
print(mask2.shape)
plt.tight_layout()
plt.show()

In [ ]:
# Same tissue image with augmentations
a = DataGenerator(train_images, train_masks, train_masks_cat, batch_size=49, augmentations=AUGMENTATIONS_TRAIN, shuffle=False)
images, masks = a.__getitem__(0)

max_images = 49
grid_width = 7
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*2, grid_height*2))

for i,(im, mask1, mask2) in enumerate(zip(images, masks[0], masks[1])):
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(im)
    # ax.imshow(mask1.squeeze(), alpha=0.4)
    ax.imshow(mask2, alpha=0.4)
    ax.axis('off')

print(mask1.shape)
print(mask2.shape)
plt.tight_layout()
plt.show()

## Defining the model

In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
# backbone = "vgg19"
backbone = "efficientnetb4"
input_shape = (256, 256, 3)
model = sm.Unet(
             backbone_name=backbone,
             input_shape=input_shape,
             )

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, show_shapes=True, dpi=330, to_file="U-Net_DD.png")

### Defining loss function for binary & multi segmentation tasks

In [ ]:
import keras.backend as K
from keras.optimizers import Adam

In [ ]:
# https://github.com/yingkaisha/keras-unet-collection
!pip install keras_unet_collection

In [ ]:
from keras_unet_collection import losses

def binary_loss(y_true, y_pred):

    loss_focal = losses.focal_tversky(y_true, y_pred, alpha=0.5, gamma=4/3)
    loss_iou = losses.iou_seg(y_true, y_pred)
    
    # (x) 
    # loss_ssim = losses.ms_ssim(y_true, y_pred, max_val=1.0, filter_size=4)
    
    return loss_focal+loss_iou #+loss_ssim

def multi_loss(y_true, y_pred):

    loss_focal = losses.focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3)
    loss_iou = losses.iou_seg(y_true, y_pred)
    
    # (x) 
    # loss_ssim = losses.ms_ssim(y_true, y_pred, max_val=1.0, filter_size=4)
    
    return loss_focal+loss_iou #+loss_ssim

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

### Compiling the model

In [ ]:
model.compile(optimizer=Adam(1e-4),
              loss={
                    "Binary_segmentation": binary_loss, 
                    "MultiClass_segmentation": multi_loss},
              metrics={
                       "Binary_segmentation": dice_coef, 
                       "MultiClass_segmentation": dice_coef})

## Train the model

### Defining some useful callbacks 

In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import datetime

from skimage import filters
from scipy.ndimage import measurements
from skimage.segmentation import watershed, mark_boundaries

# Visualize training 
class loss_history(Callback):

    def __init__(self, x=4):
        self.x = x
    
    def on_epoch_begin(self, epoch, logs={}):
        fig, ax = plt.subplots(1, 5, figsize=(18, 12))
        [axi.set_axis_off() for axi in ax.ravel()]

        ax[0].imshow(train_images[self.x])
        ax[0].set_title("Tissue Image")

        ax[1].imshow(train_masks[self.x], cmap="gray")
        ax[1].set_title("Ground Truth")

        model_sample_input = train_images[self.x].astype("float32") / 255.
        preds_train1, preds_train2 = self.model.predict(np.expand_dims(model_sample_input, axis=0), verbose=0)
        preds1 =  np.squeeze(preds_train1[0]) >= 0.5
        ax[2].imshow(preds1, cmap="gray")
        ax[2].set_title("Nuclei prediction")
        preds2 = preds_train2[0][:, :, 2] - preds_train2[0][:, :, 1] >= 0.5
        ax[3].imshow(preds2, cmap="gray")
        ax[3].set_title("Nuclei marker prediction")
        
        grad = filters.scharr(preds1)
        marker = preds1 * preds2
        marker = measurements.label(marker)[0]
        proced_pred = watershed(grad, marker, mask=preds1)
        ax[4].imshow(mark_boundaries(train_images[self.x], proced_pred, color=(0, 0, 1)))
        ax[4].set_title("Result")

        plt.tight_layout()
        plt.show()

lr_scheduler = PolynomialDecay(
    initial_learning_rate=1e-4,
    end_learning_rate=1e-7,
    decay_steps=40
)

# lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(
#     1e-4, 100, alpha=0.0, name=None
# )

class ADJUSTLR(keras.callbacks.Callback):
    def __init__ (self, model, freq, factor, verbose):
        self.model=model
        self.freq=freq
        self.factor =factor
        self.verbose=verbose
        self.adj_epoch=freq
    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 == self.adj_epoch: # adjust the learning rate
            lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
            new_lr=lr * self.factor
            self.adj_epoch +=self.freq
            if self.verbose == 1:
                print('\non epoch ',epoch + 1, ' lr was adjusted from ', lr, ' to ', new_lr)
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer


create_path("logs/Unet")
csv_log = CSVLogger('logs/Unet/Unet_log01.csv', separator=',')
model_name = f"Unet_v00_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
create_path("models/Unet")
path_to_save_model = "models/Unet/" + model_name + ".h5"
# checkpointer = ModelCheckpoint(path_to_save_model, verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1)
# reduce_lr = LearningRateScheduler(schedule=lr_scheduler)
early_stop   = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

print(model_name)

In [ ]:
callbacks = [
            #  loss_history(), 
            #  checkpointer, 
             reduce_lr, 
            #  csv_log, 
             early_stop
             ]

In [ ]:
batch_size = 8
epochs = 120

# Generators
training_generator = DataGenerator(train_images, train_masks, train_masks_cat, augmentations=AUGMENTATIONS_TRAIN, batch_size=batch_size)
validation_generator = DataGenerator(val_images, val_masks, val_masks_cat, augmentations=AUGMENTATIONS_VAL, batch_size=batch_size)

history = model.fit(training_generator,
                    validation_data=validation_generator,                        
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks)

In [ ]:
model.save(path_to_save_model)